In [3]:
import s3fs
import xarray as xr

In [4]:
bucket_src = "fcx-raw-data-temp"
# bucket_src = os.environ.get('SOURCE_BUCKET_NAME')
path_to_file="CRS/data"
# path_to_file = os.environ.get('PATH_TO_FEGS')
filename = "GOESR_CRS_L1B_20170517_v0.nc"
s3path = f"s3://{bucket_src}/{path_to_file}/{filename}"

In [5]:
s3path

's3://fcx-raw-data-temp/CRS/data/GOESR_CRS_L1B_20170517_v0.nc'

Mount S3 to local dir for direct load in xarray

In [6]:
fs = s3fs.S3FileSystem(anon=True)

In [7]:
fs.ls("fcx-raw-data-temp")

['fcx-raw-data-temp/ABI',
 'fcx-raw-data-temp/AVIRIS',
 'fcx-raw-data-temp/CPL',
 'fcx-raw-data-temp/CRS',
 'fcx-raw-data-temp/FEGS',
 'fcx-raw-data-temp/GLM',
 'fcx-raw-data-temp/ISS_LIS',
 'fcx-raw-data-temp/LIP',
 'fcx-raw-data-temp/LMA',
 'fcx-raw-data-temp/NAV_ER2',
 'fcx-raw-data-temp/S-HIS',
 'fcx-raw-data-temp/S-HIS2']

XARRAY

In [8]:
drop_these = ['gatesp', 'missing', 'incid', 'lat', 'lon',
                    'roll', 'pitch', 'track', 'height', 'head', 'evel', 'nvel',
                    'wvel', 'vacft', 'pwr', 'dop', 'frequency']

with fs.open(s3path, 'rb') as f:
    # DS = xr.load_dataset(f, engine="scipy", drop_variables=['lon', 'lat'])
    DS = xr.load_dataset(f, engine="scipy", drop_variables=drop_these)

print(DS)

<xarray.Dataset>
Dimensions:  (time: 74467, range: 321)
Coordinates:
  * time     (time) timedelta64[ns] 02:28:33.001154400 ... 07:35:23.641778400
  * range    (range) float32 1.012e+03 1.087e+03 1.162e+03 ... 2.492e+04 2.5e+04
Data variables:
    ref      (time, range) float32 -18.91 -21.36 -25.68 -17.94 ... nan nan nan
Attributes:
    year:        Year the data was collected
    date:        Day the data was collected
    title:       Cloud Radar System Data, NASA Goddard Space Flight Center
    filename:    /karlraid1a/heymsfield/GOESR/DIST/HDF/crs/GOESR_CRS_L1B_2017...
    experiment:  NOAA GOES-R
    source:      created from CRS HDF5 file, Version 0.0
    comments:    Contact: gerald.heymsfield@nasa.gov


Dataset key properties

In [9]:
DS.coords

Coordinates:
  * time     (time) timedelta64[ns] 02:28:33.001154400 ... 07:35:23.641778400
  * range    (range) float32 1.012e+03 1.087e+03 1.162e+03 ... 2.492e+04 2.5e+04

In [10]:
DS.data_vars

Data variables:
    ref      (time, range) float32 -18.91 -21.36 -25.68 -17.94 ... nan nan nan

In [11]:
DS.data_vars['ref']

<xarray.DataArray 'ref' (time: 74467, range: 321)>
array([[-18.905445, -21.356792, -25.679296, ...,        nan,        nan,
               nan],
       [-18.790478, -21.483028, -25.762894, ...,        nan,        nan,
               nan],
       [-18.927444, -21.16738 , -25.95508 , ...,        nan,        nan,
               nan],
       ...,
       [-40.323112, -36.766113, -38.658264, ...,        nan,        nan,
               nan],
       [-39.560387, -36.193375, -38.892174, ...,        nan,        nan,
               nan],
       [-38.19471 , -35.49832 , -38.108   , ...,        nan,        nan,
               nan]], dtype=float32)
Coordinates:
  * time     (time) timedelta64[ns] 02:28:33.001154400 ... 07:35:23.641778400
  * range    (range) float32 1.012e+03 1.087e+03 1.162e+03 ... 2.492e+04 2.5e+04
Attributes:
    long_name:  Radar Reflectivity
    units:      dBZ

In [12]:
DS.coords

Coordinates:
  * time     (time) timedelta64[ns] 02:28:33.001154400 ... 07:35:23.641778400
  * range    (range) float32 1.012e+03 1.087e+03 1.162e+03 ... 2.492e+04 2.5e+04

In [13]:
DS.coords['time']

<xarray.DataArray 'time' (time: 74467)>
array([ 8913001154400,  8913232040400,  8913208867200, ..., 27323186875199,
       27323413466399, 27323641778400], dtype='timedelta64[ns]')
Coordinates:
  * time     (time) timedelta64[ns] 02:28:33.001154400 ... 07:35:23.641778400
Attributes:
    long_name:  time in UTC

In [14]:
DS.coords['time'][74466]

<xarray.DataArray 'time' ()>
array(27323641778400, dtype='timedelta64[ns]')
Coordinates:
    time     timedelta64[ns] 07:35:23.641778400
Attributes:
    long_name:  time in UTC

### Taking ref. data only. Also extracting coordinates data

In [15]:
type(DS['ref'].values)
DS['ref'].values.shape

(74467, 321)

In [42]:
# DS.sel(range=slice("1011.825", "1086.775"))['ref'].values.shape
type(DS['ref'].sel(range='1011.825').values)
DS['ref'].sel(range='1011.825').values.shape
DS['ref'].sel(range='1011.825').values

array([-18.905445, -18.790478, -18.927444, ..., -40.323112, -39.560387,
       -38.19471 ], dtype=float32)

In [17]:
type(DS['time'].values)
DS['time'].values.shape

(74467,)

In [18]:
type(DS['range'].values)
DS['range'].values

array([ 1011.825,  1086.775,  1161.725,  1236.675,  1311.625,  1386.575,
        1461.525,  1536.475,  1611.425,  1686.375,  1761.325,  1836.275,
        1911.225,  1986.175,  2061.125,  2136.075,  2211.025,  2285.975,
        2360.925,  2435.875,  2510.825,  2585.775,  2660.725,  2735.675,
        2810.625,  2885.575,  2960.525,  3035.475,  3110.425,  3185.375,
        3260.325,  3335.275,  3410.225,  3485.175,  3560.125,  3635.075,
        3710.025,  3784.975,  3859.925,  3934.875,  4009.825,  4084.775,
        4159.725,  4234.675,  4309.625,  4384.575,  4459.525,  4534.475,
        4609.425,  4684.375,  4759.325,  4834.275,  4909.225,  4984.175,
        5059.125,  5134.075,  5209.025,  5283.975,  5358.925,  5433.875,
        5508.825,  5583.775,  5658.725,  5733.675,  5808.625,  5883.575,
        5958.525,  6033.475,  6108.425,  6183.375,  6258.325,  6333.275,
        6408.225,  6483.175,  6558.125,  6633.075,  6708.025,  6782.975,
        6857.925,  6932.875,  7007.825,  7082.775, 

Rough: Time manipulation

In [13]:
import numpy as np
print(np.array([8913001154400], dtype='datetime64[ns]')) # last item in time coordinate
print(np.array([27323641778400], dtype='datetime64[ns]')) # 0th item in time coordinate

['1970-01-01T02:28:33.001154400']
['1970-01-01T07:35:23.641778400']


Selecting only needed datacolumns

In [14]:
DS.data_vars['ref'].values.shape

(74467, 321)

ROUGH (Conversion to Dataframe)

In [15]:
import pandas as pd
pd.DataFrame(DS.data_vars['ref'].values)

,0,1,2,3,4,5,6,7,8,9,...,311,312,313,314,315,316,317,318,319,320
0,-18.905445,-21.356792,-25.679296,-17.936014,-16.003502,-23.358727,-15.727514,-15.907370,-19.007486,-26.448509,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-18.790478,-21.483028,-25.762894,-18.238537,-16.107655,-23.011139,-15.590458,-15.853222,-18.769602,-25.682785,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-18.927444,-21.167379,-25.955080,-18.117558,-15.938839,-22.928093,-15.487728,-15.552465,-18.293400,-25.468674,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-18.688251,-20.831541,-25.615143,-17.967514,-15.844414,-23.212997,-15.331192,-15.270306,-18.259739,-25.345592,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-18.367191,-20.563770,-25.065752,-17.985138,-15.744076,-23.093037,-15.299496,-15.306339,-18.406956,-25.098808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74462,-38.184883,-35.855206,-37.835625,-35.682014,-31.846798,-36.741299,-31.922844,-31.449917,-34.894524,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74463,-40.101379,-36.873283,-37.695450,-35.226425,-32.015217,-38.494751,-32.765282,-31.264854,-33.696205,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74464,-40.323112,-36.766113,-38.658264,-35.277237,-31.511086,-36.758156,-33.467636,-32.109810,-34.631119,-37.393093,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74465,-39.560387,-36.193375,-38.892174,-35.805649,-31.241180,-35.763390,-32.671238,-31.676991,-33.879387,-36.084679,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
DS['ref']

<xarray.DataArray 'ref' (time: 74467, range: 321)>
array([[-18.905445, -21.356792, -25.679296, ...,        nan,        nan,
               nan],
       [-18.790478, -21.483028, -25.762894, ...,        nan,        nan,
               nan],
       [-18.927444, -21.16738 , -25.95508 , ...,        nan,        nan,
               nan],
       ...,
       [-40.323112, -36.766113, -38.658264, ...,        nan,        nan,
               nan],
       [-39.560387, -36.193375, -38.892174, ...,        nan,        nan,
               nan],
       [-38.19471 , -35.49832 , -38.108   , ...,        nan,        nan,
               nan]], dtype=float32)
Coordinates:
  * time     (time) timedelta64[ns] 02:28:33.001154400 ... 07:35:23.641778400
  * range    (range) float32 1.012e+03 1.087e+03 1.162e+03 ... 2.492e+04 2.5e+04
Attributes:
    long_name:  Radar Reflectivity
    units:      dBZ

In [18]:
DF = DS.to_dataframe()

In [19]:
DF

ref
time                      range                  
0 days 02:28:33.001154400 1011.825012  -18.905445
                          1086.775024  -21.356792
                          1161.724976  -25.679296
                          1236.675049  -17.936014
                          1311.625000  -16.003502
...                                           ...
0 days 07:35:23.641778400 24696.025391        NaN
                          24770.974609        NaN
                          24845.925781        NaN
                          24920.875000        NaN
                          24995.824219        NaN

[23903907 rows x 1 columns]

In [20]:
DF[['ref']]

ref
time                      range                  
0 days 02:28:33.001154400 1011.825012  -18.905445
                          1086.775024  -21.356792
                          1161.724976  -25.679296
                          1236.675049  -17.936014
                          1311.625000  -16.003502
...                                           ...
0 days 07:35:23.641778400 24696.025391        NaN
                          24770.974609        NaN
                          24845.925781        NaN
                          24920.875000        NaN
                          24995.824219        NaN

[23903907 rows x 1 columns]

ROUGH. (to demonstrate how dataset is created and its anatomy)

In [21]:
import numpy as np
import pandas as pd
temp = 15 + 8 * np.random.randn(2, 2, 3)
temp.shape


(2, 2, 3)

In [22]:

precip = 10 * np.random.rand(2, 2, 3)
precip

array([[[5.63707269, 0.05135558, 6.06471907],
        [0.36512923, 2.50824293, 0.35934867]],

       [[1.91299343, 4.37753991, 4.28322581],
        [5.07477492, 5.38695122, 7.491222  ]]])

In [23]:

lon = [[-99.83, -99.32],
       [-99.79, -99.23]]

lat = [[42.25, 42.21],
       [42.63, 42.59]]
np.array(lat).shape

(2, 2)

In [24]:

# for real use cases, its good practice to supply array attributes such as
# units, but we won't bother here for the sake of brevity
ds = xr.Dataset(
    {
        "temperature": (["x", "y", "time"], temp),
        "precipitation": (["x", "y", "time"], precip),
    },
    coords={
        "lon": (["x", "y"], lon),
        "lat": (["x", "y"], lat),
        "time": pd.date_range("2014-09-06", periods=3),
        "reference_time": pd.Timestamp("2014-09-05"),
    },
)
ds

<xarray.Dataset>
Dimensions:         (x: 2, y: 2, time: 3)
Coordinates:
    lon             (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat             (x, y) float64 42.25 42.21 42.63 42.59
  * time            (time) datetime64[ns] 2014-09-06 2014-09-07 2014-09-08
    reference_time  datetime64[ns] 2014-09-05
Dimensions without coordinates: x, y
Data variables:
    temperature     (x, y, time) float64 25.31 22.82 27.35 ... 10.2 6.682 16.55
    precipitation   (x, y, time) float64 5.637 0.05136 6.065 ... 5.387 7.491

In [25]:
ds.data_vars['precipitation']

<xarray.DataArray 'precipitation' (x: 2, y: 2, time: 3)>
array([[[5.63707269, 0.05135558, 6.06471907],
        [0.36512923, 2.50824293, 0.35934867]],

       [[1.91299343, 4.37753991, 4.28322581],
        [5.07477492, 5.38695122, 7.491222  ]]])
Coordinates:
    lon             (x, y) float64 -99.83 -99.32 -99.79 -99.23
    lat             (x, y) float64 42.25 42.21 42.63 42.59
  * time            (time) datetime64[ns] 2014-09-06 2014-09-07 2014-09-08
    reference_time  datetime64[ns] 2014-09-05
Dimensions without coordinates: x, y

In [26]:
precip

array([[[5.63707269, 0.05135558, 6.06471907],
        [0.36512923, 2.50824293, 0.35934867]],

       [[1.91299343, 4.37753991, 4.28322581],
        [5.07477492, 5.38695122, 7.491222  ]]])

In [27]:
cat = [[42.25, 42.21, 1],
       [42.63, 42.59, 1 ],
       [2,     2,     2]]
np.array(cat).shape 
# this means there are two dimensions, i.e. features VS no of data table. 
# There can be many features and there can be surely many data points for those features

(3, 3)

In [28]:
catss = [[[42.25, 42.21, 1],
        [42.63, 42.59, 1 ],
        [2,     2,     2]],
 
        [[42.25, 42.21, 1],
        [42.63, 42.59, 1 ],
        [2,     2,     2]]]
np.array(catss).shape 
# this means there are three dimensions, i.e. features VS no of data table. And on top of that, test_group(say) 
# There can be many features and there can be surely many data points for those features
# there are many test_groups conducted on various geographical locations, each seperated out say g1 and g2

(2, 3, 3)

Numpy array difference

In [29]:
source = ['a', 'b', 'c', 'dog']
target = ['dog']
np.setdiff1d(source, target)

array(['a', 'b', 'c'], dtype='<U3')